# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [50]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [51]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [52]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df=df.dropna()
df=df.drop_duplicates()
df = df.sort_index()
df.tail()

,fng_value
date,
2019-07-25,42
2019-07-26,24
2019-07-27,47
2019-07-28,16
2019-07-29,19


In [53]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [54]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2=df2.dropna()
df2=df2.drop_duplicates()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [55]:
type(df2.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [56]:
# Join the data into a single DataFrame
newdf = pd.concat([df,df2], join="outer", axis="columns")
newdf.tail()

,fng_value,Close
2019-07-25,42.0,9882.429688
2019-07-26,24.0,9847.450195
2019-07-27,47.0,9478.320313
2019-07-28,16.0,9531.769531
2019-07-29,19.0,9529.889648


In [57]:
newdf.dropna(inplace = True)
newdf.head()

,fng_value,Close
2018-02-06,8.0,7701.250000
2018-07-30,53.0,8176.060059
2018-10-31,32.0,6342.609863
2018-11-12,52.0,6375.080078
2018-11-25,9.0,4004.149902


In [58]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [59]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 5

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(newdf, window_size, feature_column, target_column)

In [60]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split -1]
y_test = y[split:]

In [61]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.


scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)


In [62]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.        ]
  [0.51162791]
  [0.26744186]
  [0.5       ]
  [0.        ]]

 [[0.51724138]
  [0.26744186]
  [0.5       ]
  [0.        ]
  [0.03488372]]

 [[0.27586207]
  [0.5       ]
  [0.        ]
  [0.03488372]
  [0.01162791]]

 [[0.50574713]
  [0.        ]
  [0.03488372]
  [0.01162791]
  [0.02325581]]

 [[0.01149425]
  [0.03488372]
  [0.01162791]
  [0.02325581]
  [0.04651163]]] 

X_test sample values:
[[[0.77011494]
  [0.84883721]
  [0.8372093 ]
  [0.8255814 ]
  [0.90697674]]

 [[0.85057471]
  [0.8372093 ]
  [0.8255814 ]
  [0.90697674]
  [1.        ]]

 [[0.83908046]
  [0.8255814 ]
  [0.90697674]
  [1.        ]
  [0.96511628]]

 [[0.82758621]
  [0.90697674]
  [1.        ]
  [0.96511628]
  [0.80232558]]

 [[0.90804598]
  [1.        ]
  [0.96511628]
  [0.80232558]
  [0.62790698]]]


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [64]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# YOUR CODE HERE!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [65]:
#Define LSTRM model
model = Sequential()

#Layer 1


In [66]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam"
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
batch_size = 1000
model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=batch_size,
    verbose=1,
)


Epoch 1/10
1/1 [==============================] - 0s 177ms/step - loss: 0.5151
Epoch 2/10
1/1 [==============================] - 0s 6ms/step - loss: 0.5151
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.5151
Epoch 4/10
1/1 [==============================] - 0s 7ms/step - loss: 0.5151
Epoch 5/10
1/1 [==============================] - 0s 15ms/step - loss: 0.5151
Epoch 6/10
1/1 [==============================] - 0s 11ms/step - loss: 0.5151
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5151
Epoch 8/10
1/1 [==============================] - 0s 7ms/step - loss: 0.5151
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 0.5151
Epoch 10/10
1/1 [==============================] - 0s 21ms/step - loss: 0.5151


---

In [68]:
# Summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [69]:
# Evaluate the model
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 178ms/step - loss: 0.6980


0.6979954242706299

In [70]:
# Make some predictions
predicted = model.predict(X_test)
predicted

array([[[0.77011496],
        [0.8488372 ],
        [0.8372093 ],
        [0.8255814 ],
        [0.90697676]],

       [[0.85057473],
        [0.8372093 ],
        [0.8255814 ],
        [0.90697676],
        [1.        ]],

       [[0.83908045],
        [0.8255814 ],
        [0.90697676],
        [1.        ],
        [0.96511626]],

       [[0.82758623],
        [0.90697676],
        [1.        ],
        [0.96511626],
        [0.8023256 ]],

       [[0.90804595],
        [1.        ],
        [0.96511626],
        [0.8023256 ],
        [0.627907  ]],

       [[1.        ],
        [0.96511626],
        [0.8023256 ],
        [0.627907  ],
        [0.81395346]],

       [[0.9655172 ],
        [0.8023256 ],
        [0.627907  ],
        [0.81395346],
        [0.7790698 ]],

       [[0.8045977 ],
        [0.627907  ],
        [0.81395346],
        [0.7790698 ],
        [0.73255813]],

       [[0.6321839 ],
        [0.81395346],
        [0.7790698 ],
        [0.73255813],
        [0.67441

In [71]:
#y_test


In [72]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [46]:
#predicted.reshape(-1,1)
predicted_prices

array([[10687.815 , 11449.908 , 11337.341 , 11224.773 , 12012.743 ],
       [11466.7295, 11337.341 , 11224.773 , 12012.743 , 12913.281 ],
       [11355.456 , 11224.773 , 12012.743 , 12913.281 , 12575.579 ],
       [11244.182 , 12012.743 , 12913.281 , 12575.579 , 10999.641 ],
       [12023.094 , 12913.281 , 12575.579 , 10999.641 ,  9311.133 ],
       [12913.281 , 12575.579 , 10999.641 ,  9311.133 , 11112.206 ],
       [12579.46  , 10999.641 ,  9311.133 , 11112.206 , 10774.506 ],
       [11021.635 ,  9311.133 , 11112.206 , 10774.506 , 10324.237 ],
       [ 9352.537 , 11112.206 , 10774.506 , 10324.237 ,  9761.401 ],
       [11132.908 , 10774.506 , 10324.237 ,  9761.401 , 10549.372 ],
       [10799.09  , 10324.237 ,  9761.401 , 10549.372 , 11675.042 ],
       [10353.997 ,  9761.401 , 10549.372 , 11675.042 , 11562.475 ],
       [ 9797.631 , 10549.372 , 11675.042 , 11562.475 ,  9198.566 ],
       [10576.542 , 11675.042 , 11562.475 ,  9198.566 ,  5934.1206],
       [11689.274 , 11562.475 ,  9

In [47]:
newdf.index[len(real_prices):]

DatetimeIndex(['2019-02-08', '2019-02-14', '2019-02-17', '2019-02-28',
               '2019-03-04', '2019-03-05', '2019-03-16', '2019-03-20',
               '2019-03-26', '2019-03-27', '2019-03-28', '2019-03-31',
               '2019-04-05', '2019-04-25', '2019-04-26', '2019-05-01',
               '2019-05-02', '2019-05-06', '2019-05-15', '2019-05-21',
               '2019-05-24', '2019-05-25', '2019-05-27', '2019-05-29',
               '2019-05-30', '2019-06-03', '2019-06-07', '2019-06-10',
               '2019-06-12', '2019-06-15', '2019-06-19', '2019-06-20',
               '2019-06-24', '2019-06-25', '2019-06-26', '2019-06-27',
               '2019-06-30', '2019-07-02', '2019-07-03', '2019-07-04',
               '2019-07-06', '2019-07-07', '2019-07-08', '2019-07-09',
               '2019-07-10', '2019-07-11', '2019-07-12', '2019-07-13',
               '2019-07-14', '2019-07-20', '2019-07-23', '2019-07-24',
               '2019-07-25', '2019-07-26', '2019-07-27', '2019-07-28',
      

In [48]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
},index=newdf.index[-len(real_prices):] )
stocks.head()

ValueError: Shape of passed values is (110, 2), indices imply (22, 2)

In [101]:
# Plot the real vs predicted values as a line chart
stocks.plot()


NameError: name 'stocks' is not defined